## 0. Introduction

The aim of this lab is to get familiar with **Face Recognition** using  **eigenfaces**, as described in Turk & Pentland (1991).


1.   This lab is the second course-work activity **Assignment 2**
2.   A report answering the <font color = 'red'>**questions in</font><font color = "maroon"> red**</font> should be submitted on QMplus along with the completed Notebook.
3. The report should be a separate file in **pdf format** (so **NOT** *doc, docx, notebook* etc.), well identified with your name, student number, assignment number (for instance, Assignment 1), module code.
4. Make sure that **any figures or code** you comment on, are **included in the report**.
5. No other means of submission other than the appropriate QM+ link is acceptable at any time (so NO email attachments, etc.)
6. **PLAGIARISM** <ins>is an irreversible non-negotiable failure in the course</ins> (if in doubt of what constitutes plagiarism, ask!).


Bib:

Turk, M. and Pentland, A., 1991. Eigenfaces for recognition. *Journal of cognitive neuroscience*, 3(1), pp.71-86.

In [ ]:
from torch import pca_lowrank
from sklearn.model_selection import train_test_split
from sklearn import datasets
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import cv2

We will use the [Olivetti faces dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html#sklearn.datasets.fetch_olivetti_faces) from AT&T.

The dataset consists of 400, 64 x 64 grey-scale images. We will split this in train and test sets, where 75\% is used in the training and the remaining in the test set.

In [ ]:
faces = datasets.fetch_olivetti_faces()
fig = plt.figure(figsize=(8, 6))
for i in range(15):
    ax = fig.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(faces.images[i], cmap=plt.cm.bone)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    faces.data,
    faces.target,
    random_state=0
    )

print(X_train.shape, X_test.shape) 
# 在X_train中存储的数据代表Olivetti人脸数据库中图像的像素值，这些图像用于训练机器学习模型进行人脸识别或其他相关任务。每个图像被展平成一个一维数组，其中的每个元素对应一个像素值。这些像素值一般是归一化过的，范围在0到1之间，代表图像的灰度级别

## 1. Face Recognition using Eigenfaces

Task 1: Construct the mean image and the covariance matrix of the training image set. <font color = 'red'> Explain the method implemented and display the mean image.</font> [4 marks]

In [ ]:
### YOUR CODE
# calculate the mean face of the train set
# calculate the covariance matrix
# display the mean face (hint: you will need to reshape the mean)
mean_face = np.mean(X_train, axis=0)
cov_matrix = np.cov(X_train.T) # 特征相关性：协方差矩阵可以帮助我们了解训练集中的特征之间的相关性。协方差的数值表示了两个特征之间的线性关系，如果协方差为正，表示两个特征呈正相关，如果为负，则呈负相关，如果接近于零，则表示两个特征之间不存在线性相关性。特征提取：在人脸识别中，协方差矩阵通常被用于进行特征提取。例如，通过对协方差矩阵进行特征值分解，我们可以得到最重要的特征向量（即主成分），然后利用这些主成分来表示数据，从而实现降维和特征提取的目的。
mean_face_reshaped = mean_face.reshape((64, 64))

plt.figure(figsize=(4, 4))
plt.imshow(mean_face_reshaped, cmap='gray')
plt.title("Mean Face")
plt.xticks([])
plt.yticks([])
plt.show()

mean_face.shape, cov_matrix.shape

Recall from the lectures, to solve PCA we need the eigenvectors of
the covariance matrix so that: $[U,V]=eig(X,X.T)$. However, faster convergence is achieved without using explicit covariance so:  $[U,S,V]=svd(X)$

where:
* $U$ is a $m \times q$ matrix
* $S$ is a $q$ vector
* $V$ is a $n \times q$ matrix

Pytorch provides a lower level method that allows us to explore $[U, S, V]$ through [pca_lowrank](https://pytorch.org/docs/stable/generated/torch.pca_lowrank.html)

Task 2: Compute the eigenfaces of the training set, using [pca_lowrank](https://pytorch.org/docs/stable/generated/torch.pca_lowrank.html)
.  <font color = 'red'> Explain eigenfaces and display the 20 first eigenfaces</font> [4 marks]


In [ ]:
### YOUR CODE HERE
import torch

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)

# Compute the low-rank approximation of the PCA
U, S, V = pca_lowrank(X_train_tensor, 20) # U, S, V分别代表主成分分析（PCA）的低秩近似中的三个重要矩阵。U是左奇异向量矩阵，表示数据中的特征方向；S是对角矩阵，包含奇异值，表示每个特征方向的重要性或方差；V是右奇异向量矩阵，通常用于降维后的数据重构

num_eigenfaces = V.shape[1]

fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()
for i in range(num_eigenfaces):
    eigenface_reshaped = V[:, i].reshape(64, 64).detach().numpy()
    axes[i].imshow(eigenface_reshaped, cmap='gray')
    axes[i].set_title(f'Eigenface {i+1}')
    axes[i].axis('off')

plt.show()

$A$ is a data matrix with m samples and n features

$V$ columns represent the principal directions

$S^2 / (m - 1)$ contains the eigenvalues of $A.T * A / (m - 1)$

`matmul(A, V[:, :k])` projects data to the first k principal components.

Task 3:  Project both training images and testing images onto the first 20 eigenfaces.


---


Compute the distance from the projected test images to the projected training images.

<font color = 'red'>
Display the top 6 best matched training images for the first 6 test images.
</font>

<font color = 'red'>
Compute the recognition rate using 20 eigenfaces (predicted class is the class of the nearest training set image).
</font>

<font color = 'red'>
Investigate the effect of using different number of eigenfaces for recognition (e.g. plot the recognition rate against the number of eigenfaces).
</font> [7 marks]

In [ ]:
### YOUR CODE HERE
# Step 1: Project all images onto the first 20 eigenfaces.
#     Hint: this is regulated by the number of components

k = 20  # Number of eigenfaces to use for projection
X_train_proj = np.dot(X_train, V[:, :k])   # 使用np.dot进行PCA投影是因为我们需要将数据转换到这些主成分构成的新空间中，np.dot用于计算数据点和主成分之间的点积，从而实现这种转换。这样可以降维，同时保留数据的主要变化信息。
X_test_proj = np.dot(X_test, V[:, :k])
print('X_train_proj.shape', X_train_proj.shape)
print('X_test_proj.shape', X_test_proj.shape)
# X_train_proj.shape (300, 20)表示变量X_train_proj是一个形状为300行20列的矩阵。这里，300代表样本的数量，而20代表经过主成分分析（PCA）降维后保留的特征数量。每行代表一个样本，每列代表一个降维后的特征。

In [ ]:
# Step 2: Compute the distance from the projected test images to the projected
# training.

def euclidean_distance(v1, v2):
  d = 0
  ### fill in the method
  d=np.sqrt(np.sum((v1 - v2) ** 2))
  return d

# Find the distance between test and training projections
distances = np.array([[euclidean_distance(test, train) for train in X_train_proj] for test in X_test_proj])

distances.shape #shape为(100, 300) 每行代表一个测试样本与所有训练样本的距离，每列代表所有测试样本与一个训练样本的距离。

In [ ]:
# Step 3: Display the top 6 best matched images for the first 6 test images
top_match_num=6
# np.argsort是一个NumPy库中的函数，用于返回数组元素从小到大排序的索引。它不会改变原数组，而是返回一个新的数组，包含按排序顺序的索引
top_matches = np.argsort(distances, axis=1)[:, :top_match_num] # 存储了每个测试图像与训练集图像之间距离最小的几个索引
print('top_matches',top_matches.shape)

fig, axes = plt.subplots(nrows=top_match_num, ncols=top_match_num + 1, figsize=(20, 10))

for i in range(top_match_num):
    # Display the test image
    axes[i, 0].imshow(X_test[i].reshape((64, 64)), cmap='gray')
    axes[i, 0].set_title('Test Image {}'.format(i + 1))
    axes[i, 0].axis('off')

    # Display the top 6 matching training images
    for j in range(top_match_num):
        match_idx = top_matches[i, j]
        axes[i, j + 1].imshow(X_train[match_idx].reshape((64, 64)), cmap='gray')
        axes[i, j + 1].set_title('Match {}'.format(j + 1))
        axes[i, j + 1].axis('off')

plt.show()

In [ ]:
# Step 4: Assign a class to each image on the test set (class of test image is
# the class of it's nearest neighbour in the train set)
# and calculate the recognition rate

# the class of it's nearest neighbour in the train set
predicted_classes = y_train[top_matches[:, 0]]
print(y_train.shape)
print(top_matches[:, 0].shape)

# Calculate recognition rate
accuracy = np.mean(predicted_classes == y_test) #通过比较predicted_classes和y_test中的每个元素是否相等（返回一个布尔数组），然后计算这个布尔数组中True的比例，从而得到准确率


print('predicted_classes',predicted_classes.shape)
print('accuracy',accuracy)

In [ ]:
# Step 5: investigate effect of number of eigenfaces for recognition rate
num_eigenfaces = range(1, 21)
recognition_rates = []

for k in num_eigenfaces:
    X_train_proj_k = np.dot(X_train, V[:, :k])
    X_test_proj_k = np.dot(X_test, V[:, :k])
    distances_k = np.array([[euclidean_distance(test, train) for train in X_train_proj_k] for test in X_test_proj_k])
    top_match_k = np.argsort(distances_k, axis=1)[:, 0]
    predicted_classes_k = y_train[top_match_k]
    accuracy_k = np.mean(predicted_classes_k == y_test)
    recognition_rates.append(accuracy_k)

print('recognition_rates',recognition_rates)

# Plotting the recognition rate against the number of eigenfaces
plt.figure(figsize=(10, 6))
plt.plot(num_eigenfaces, recognition_rates, marker='o')
plt.title('the recognition rate against the number of eigenfaces')
plt.xlabel('Number of Eigenfaces')
plt.ylabel('Recognition Rate')
plt.xticks(num_eigenfaces)
plt.grid(True)
plt.show()

Task 4: So far we have implemented a KNN classifier with K=1. <font color = 'red'> Investigate the effect of K in K-Nearest-Neighbour (KNN) classifier. Plot the average recognition rate against K).</font> [5 marks]

In [ ]:
### YOUR CODE HERE

def knn_classify(distances, y_train, k_num):
    top_match_k = np.argsort(distances, axis=1)[:, k_num]
    predicted_classes = y_train[top_match_k]
    return predicted_classes

# Range of k values to test
k_values = range(1, 21)
recognition_rates = [] 

for i in k_values:
    predicted_classes = knn_classify(distances, y_train, i)
    recognition_rate = np.mean(predicted_classes == y_test)
    recognition_rates.append(recognition_rate)

print('recognition_rates',recognition_rates)

# Plot the recognition rate against k
plt.figure(figsize=(10, 6))
plt.plot(k_values, recognition_rates, marker='o')
plt.xlabel('K in KNN')
plt.ylabel('Recognition Rate')
plt.title('Recognition Rate vs. K in KNN')
plt.xticks(k_values)
plt.grid(True)
plt.show()


## 2. Handing in

Create a folder that will contain:

* A .pdf report that contains the answers to the exercises
* The programs files

Create a .zip file and submit electronically on QMplus.

IMPORTANT: Plagiarism (copying from other students, or copying the work of others without proper referencing) is cheating, and will not be tolerated.

IF TWO “FOLDERS” ARE FOUND TO CONTAIN IDENTICAL MATERIAL, BOTH WILL BE GIVEN A MARK OF ZERO.
